In [13]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [14]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Krmiljenje smeri gibanja avtonomnega podvodnega vozila (AUV) 

Dinamiko smeri gibanja $\psi$ avtonomnega podvodnega vozila (AUV), na repu katerega so nameščene pogonske črpalke, ki proizvajajo rotacijo, lahko popišemo z

$$
J_z\ddot{\psi} = −DV\dot{\psi} + P\alpha,
$$

kjer so $ J_z = 3.75 $ kg $ \ text {m}^2 $ vztrajnostni moment vozila okoli osi jaw/smeri gibanja, $ D = 25 $ Nms koeficient zračnega upora, $ V $ trenutna hitrost vozila (v m /s), $ P = 0.32 $ Nms koeficient pogonskih črpalk, in $ \ alpha $ rotacijska hitrost propelerja pogonskih črpalk (v rad/s). Propeler znotraj pogonskih črpalk je aktuiran z elektromotorjem, katerega hitrost mora biti znotraj $ \ pm100 $ obratov na minuto (RPM). Dinamiko elektromotorja lahko modeliramo z

$$
M(s)=\frac{\alpha(s)}{u(s)}=0.45\frac{\omega_n^2}{s^2+2\omega_n\xi s +\omega_n^2}
$$

pri čemer velja $\omega_n = 9$ in $\xi = 0.8$, $u(𝑠)$ pa predstavlja napajalno napetost motorja. Hitrost gibanja je merjena z elektronskim žirokompasom.

Maksimalna dovoljena napetost $u$ v stacionarnem stanju je (ob upoštevanju omejitve obratov motorja ter teorema končne vrednosti)

$$
u_{\text{max}} = \frac{\alpha_{\text{max}}}{M(s)|_{s=0}} = \frac{\pm100 \frac{2\pi}{60}}{0.45} \approx \pm 23.3 \text{V} \, .
$$

Ob upoštevanju hitrostnega skoka hitrosti $V$ v velikosti 1 m/s in koračne vhodne funkcije, ki je enaka polovici največje vrednosti $\dot{\psi}$, ki je dosegljiva v stacionarnem stanju, želimo načrtovati krmilnik, ki bo ustrezal naslednjihm zahtevam:
- čas ustalitve krajši od 5 s,
- brez prenihaja,
- brez odstopka v stacionarnem stanju v odzivu na hitrostni vhod v obliki koračne funkcije,
- maksimalna hitrost propelerja pogonskih črpalk ni presežena.

Najprej moramo realizirati dinamiko električnega motorja in jo nato pripeti k dinamiki smeri gibanja AUV. Možna rešitev, ob tem, da velja $z=\begin{bmatrix} z_1 & z_2 \end{bmatrix}^T$, je

\begin{cases}
    \dot{z} = \underbrace{\begin{bmatrix} 0 & 1 \\ -\omega_n^2 & -2\xi\omega_n \end{bmatrix}}_{A_m}z + \begin{bmatrix} 0 \\ 0.45 \end{bmatrix}u \\
    \alpha = \begin{bmatrix} \omega_n^2 & 0 \end{bmatrix}z
\end{cases}

tako lahko celotno dinamiko v obliki prostora stanj z $x = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}^T = \begin{bmatrix} \dot{\psi} & z_1 & z_2 \end{bmatrix}^T$, zapišemo kot:

\begin{cases}
    \dot{x} = \begin{bmatrix} -\frac{DV}{J_z} & \frac{P}{J_z}\omega_n^2 & 0 \\ 0 & 0 & 1 \\ 0 & -\omega_n^2 & -2\xi\omega_n \end{bmatrix}x + \begin{bmatrix} 0 \\ 0 \\ 0.45 \end{bmatrix}u \\
    y = \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}x \, .
\end{cases}

$\dot{\psi}_{\text{max}} = \frac{P}{DV}\alpha_{\text{max}} = \frac{P}{DV}100 \frac{2\pi}{60} \approx 0.134$ rad/s.

### Načrtovanje regulatorja
#### Načrtovanje krmilnika

Da zadostimo zahtevi po ničelnem odstopku v stacionarnem stanju, dodamo novo spremenljivko stanj:
$$
\dot{x_4} = \dot{\psi}-y_d = x_1 - y_d
$$
Rezultat je naslednji razširjen sistem
\begin{cases}
    \dot{x_a} = \begin{bmatrix} -\frac{DV}{J_z} & \frac{P}{J_z}\omega_n^2 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & -\omega_n^2 & -2\xi\omega_n & 0 \\ 1 & 0 & 0 & 0 \end{bmatrix}x_a + \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 0.45 & 0 \\ 0 & -1 \end{bmatrix}\begin{bmatrix} u \\ y_d \end{bmatrix} \\
    y_a = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}x_a,
\end{cases}

kjer velja $x_a = \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}^T = \begin{bmatrix} \dot{\psi} & z_1 & z_2 & x_4 \end{bmatrix}^T$, drugi izhod pa je dodan zaradi tega, da ohranimo sistem spoznaven. Z namenom, da poizkušamo omejiti amplitudo vhoda $u$ in s tem tudi $\alpha$, razporedimo zgolj pol, ki je povezan s stanjem $x_4$. Izbrani poli so $-\frac{DV}{J_z} \approx -6.67$, $-\xi\omega_n \pm \sqrt{1-\xi^2}\omega_n = -7.2 \pm -5.4i$ in $-0.7$.

#### Načrtovanje spoznavalnika
Ker so meritve $x_1$ in $x_4$ na vojo, potrebujemo zgolj oceno stanj $x_1$ in $x_2$. Spoznavalnik načrtujemo s celotnim sistemom velikosti $4\times4$, pole spoznavalnika pa postavimo v $-15$.

### Kako upravljati s tem interaktivnim primerom?
- V primeru, da velja $V = 1.5$ m/s, poizkusi slediti zahtevam na način, da spremeniš lokacije polov krmilnika.
- Poizkušaj skrajšati čas ustalitve pod 3 s, pri čemer je dovoljen maksimalni prenihaj v vrednosti 10%.

In [15]:
# Preparatory cell

X0 = numpy.matrix('0.0; 0.0; 0.0; 0.0')
K = numpy.matrix([0,0,0,0])
L = numpy.matrix([[0,0],[0,0],[0,0],[0,0]])

X0w = matrixWidget(4,1)
X0w.setM(X0)
Kw = matrixWidget(1,4)
Kw.setM(K)
Lw = matrixWidget(4,2)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig4c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-6.67])) 
eig2c.setM(numpy.matrix([[-7.2],[-5.4]]))
eig3c.setM(numpy.matrix([-0.7]))
eig4c.setM(numpy.matrix([[-1.],[-1.]]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig4o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-15.])) 
eig2o.setM(numpy.matrix([[-15.],[0.]]))
eig3o.setM(numpy.matrix([-15.]))
eig4o.setM(numpy.matrix([[-15.],[0.]]))

In [16]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Nastavi K in L', 'Nastavi lastne vrednosti'],
    value= 'Nastavi lastne vrednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
selec = widgets.Dropdown(
    options= ['brez kompleksnih lastnih vrednosti', 'dve kompleksni lastni vrednosti', 'štiri kompleksne lastne vrednosti'],
    value= 'dve kompleksni lastni vrednosti',
    description='Kompleksne lastne vrednosti krmilnika:',
    disabled=False
)
seleo = widgets.Dropdown(
    options= ['brez kompleksnih lastnih vrednosti', 'dve kompleksni lastni vrednosti', 'štiri kompleksne lastne vrednosti'],
    value= 'brez kompleksnih lastnih vrednosti',
    description='Kompleksne lastne vrednosti spoznavalnika:',
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzna funkcija', 'koračna funkcija', 'sinusoidna funkcija', 'kvadratni val'],
    value='koračna funkcija',
    description='Vhod:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=0.134,
    min=0,
    max=0.5,
    step=0.001,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
v = widgets.FloatSlider(
    value=1,
    min=0.5,
    max=4,
    step=0.1,
    description=r'$V$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Perioda: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

simTime = widgets.FloatText(
    value=10,
    description='',
    disabled=False
)

In [17]:
# Support functions

def eigen_choice(selec,seleo):
    if selec == 'brez kompleksnih lastnih vrednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = True
        eigc = 0
    if seleo == 'brez kompleksnih lastnih vrednosti':
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = True
        eigo = 0
    if selec == 'dve kompleksni lastni vrednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = True
        eig4c.children[1].children[0].disabled = True
        eigc = 2
    if seleo == 'dve kompleksni lastni vrednosti':
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = True
        eig4o.children[1].children[0].disabled = True
        eigo = 2
    if selec == 'štiri kompleksne lastne vrednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = True
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = False
        eigc = 4
    if seleo == 'štiri kompleksne lastne vrednosti':
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = True
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = False
        eigo = 4
    return eigc, eigo

def method_choice(selm):
    if selm == 'Nastavi K in L':
        method = 1
        selec.disabled = True
        seleo.disabled = True
    if selm == 'Nastavi lastne vrednosti':
        method = 2
        selec.disabled = False
        seleo.disabled = False
    return method

In [18]:
D = 25
P = 0.32
V = 1
Jz = 3.75

A = numpy.matrix([[-D*V/Jz, P/Jz*9**2, 0, 0],
                  [0, 0, 1, 0],
                  [0, -9**2, -2*0.8*9, 0],
                  [1, 0, 0, 0]])
Bu = numpy.matrix([[0],[0],[0.45],[0]])
Bref = numpy.matrix([[0],[0],[0],[-1]])
C = numpy.matrix([[1,0,0,0],[0,0,0,1]])

def main_callback2(v, X0w, K, L, eig1c, eig2c, eig3c, eig4c, eig1o, eig2o, eig3o, eig4o, u, period, selm, selec, seleo, selu, simTime, DW):
    eigc, eigo = eigen_choice(selec,seleo)
    method = method_choice(selm)
    
    A = numpy.matrix([[-D*v/Jz, P/Jz*9**2, 0, 0],
                  [0, 0, 1, 0],
                  [0, -9**2, -2*0.8*9, 0],
                  [1, 0, 0, 0]])
    
    if method == 1:
        solc = numpy.linalg.eig(A-Bu*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        #for better numerical stability of place
        if eig1c[0,0]==eig2c[0,0] or eig1c[0,0]==eig3c[0,0] or eig1c[0,0]==eig4c[0,0]:
            eig1c[0,0] *= 1.01
        if eig2c[0,0]==eig3c[0,0] or eig2c[0,0]==eig4c[0,0]:
            eig3c[0,0] *= 1.015
        if eig1o[0,0]==eig2o[0,0] or eig1o[0,0]==eig3o[0,0] or eig1o[0,0]==eig4o[0,0]:
            eig1o[0,0] *= 1.01
        if eig2o[0,0]==eig3o[0,0] or eig2o[0,0]==eig4o[0,0]:
            eig3o[0,0] *= 1.015
            
        if eigc == 0:
            K = control.acker(A, Bu, [eig1c[0,0], eig2c[0,0], eig3c[0,0], eig4c[0,0]])
            Kw.setM(K)
        if eigc == 2:
            K = control.acker(A, Bu, [eig3c[0,0],
                                      eig1c[0,0],
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigc == 4:
            K = control.acker(A, Bu, [numpy.complex(eig4c[0,0], eig4c[1,0]), 
                                      numpy.complex(eig4c[0,0],-eig4c[1,0]),
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigo == 0:
            L = control.place(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0], eig4o[0,0]]).T
            Lw.setM(L)
        if eigo == 2:
            L = control.place(A.T, C.T, [eig3o[0,0],
                                         eig1o[0,0],
                                         numpy.complex(eig2o[0,0], eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
        if eigo == 4:
            L = control.place(A.T, C.T, [numpy.complex(eig4o[0,0], eig4o[1,0]), 
                                      numpy.complex(eig4o[0,0],-eig4o[1,0]),
                                      numpy.complex(eig2o[0,0], eig2o[1,0]), 
                                      numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    sys = sss(A,numpy.hstack((Bu,Bref)),[[1,0,0,0],[0,0,0,1],[0,0,0,0]],[[0,0],[0,0],[0,1]])
    syse = sss(A-L*C,numpy.hstack((Bu,Bref,L)),numpy.eye(4),numpy.zeros((4,4)))
    sysc = sss(0,[0,0,0,0],0,-K)
    sys_append = control.append(sys,syse,sysc)
    try:
        sys_CL = control.connect(sys_append,
                                 [[1,8],[3,8],[5,1],[6,2],[7,4],[8,5],[9,6],[10,7],[4,3]],
                                 [2],
                                 [1,8])
    except:
        sys_CL = control.connect(sys_append,
                                 [[1,8],[3,8],[5,1],[6,2],[7,4],[8,5],[9,6],[10,7],[4,3]],
                                 [2],
                                 [1,8])

    X0w1 = numpy.zeros((8,1))
    X0w1[4,0] = X0w[0,0]
    X0w1[5,0] = X0w[1,0]
    X0w1[6,0] = X0w[2,0]
    X0w1[7,0] = X0w[3,0]
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulzna funkcija': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'koračna funkcija':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'sinusoidna funkcija':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'kvadratni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    
    try:
        step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
        print('Informacije o odzivu sistema: \n\tČas vzpona [s] =',step_info_dict['RiseTime'],'\n\tČas ustalitve (5%) [s] =',step_info_dict['SettlingTime'],'\n\tPrenihaj [%]=',step_info_dict['Overshoot'])
        print('Maksimalna vrednost alpha (delež od 100 RPM)=', max(abs(xout[1]*9**2*60/2/numpy.pi))/(100)*100)
        print('Maksimalna vrednost u (delež od 23.3 V)=', max(abs(yout[1]))/(23.3)*100)
    except:
        print("Napaka v izračunu informacij o odzivu sistema.")
    
    fig = plt.figure(num='Simulacija 1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Odziv sistema')
    plt.ylabel('Izhod')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Referenca'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Vhod')
    plt.ylabel('$u$')
    plt.plot(T,yout[1])
    plt.plot(T,[23.3 for i in range(len(T))],'r--')
    plt.plot(T,[-23.3 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Odzivi stanj')
    plt.ylabel('Stanja')
    plt.plot(T,xout[0],
             T,xout[1],
             T,xout[2],
             T,xout[3],
             T,[100*2*numpy.pi/60/9**2 for i in range(len(T))],'r--',
             T,[-100*2*numpy.pi/60/9**2 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$','$x_{4}$','+lim $x_2$','-lim $x_2$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Napaka ocene stanj')
    plt.ylabel('Napaka ocene stanj')
    plt.plot(T,xout[4]-xout[0])
    plt.plot(T,xout[5]-xout[1])
    plt.plot(T,xout[6]-xout[2])
    plt.plot(T,xout[7]-xout[3])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$','$e_{2}$','$e_{3}$','$e_{4}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          selec,
                                          seleo,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                                        widgets.Label('Lastne vrednosti:',border=3),
                                                        widgets.HBox([eig1c, 
                                                                      eig2c, 
                                                                      eig3c,
                                                                      eig4c])])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.HBox([widgets.Label('L:',border=3), Lw, widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Lastne vrednosti:',border=3), 
                                          eig1o, 
                                          eig2o,
                                          eig3o, 
                                          eig4o,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                                          widgets.Label(' ',border=3),
                                                       widgets.HBox([
                                          widgets.VBox([widgets.Label('Simulacijski čas [s]:',border=3)]),
                                          widgets.VBox([simTime])])]),
                                          widgets.Label(' ',border=3)]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('Referenca [rad]:',border=3),
                                          u,
                                          v]),
                            widgets.HBox([period,START])])
out2 = widgets.interactive_output(main_callback2, {'v':v, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                   'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig4c':eig4c, 
                                                   'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 'eig4o':eig4o,  
                                                   'u':u, 'period':period, 'selm':selm, 'selec':selec, 'seleo':seleo, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '880px'
display(out2, alltogether2)

Output(layout=Layout(height='880px'))